In [2]:
import torch
import torchvision
import json
import pandas as pd
from healthpricefinder.utils import iterate_for_df
import ijson
import os


In [3]:
# !pip install -e ../../.

In [4]:
df_row_base = {"negotiation_arrangement": [None], "name": [None], "billing_code_type": [None], "billing_code_type_version": [None], "billing_code": [None],
              "description": [None], "negotiated_rate": [None], "expiration": [None], "provider_references": [None], "negotiated_type": [None], "billing_class": [None]}


## Extract provider reference id to NPI relationship

In [32]:
provider_reference = next(f)

In [44]:
prov_dict = provider_reference[:3090816]
prov_dict += "}"

In [64]:
providers = json.loads(prov_dict)["provider_references"]
provider_df = pd.DataFrame()
for p in providers:
    provider_dict = {"provider_id": [], "npi_list": [], "tin": [], "tin_type": []}
    provider_dict["provider_id"].append(p["provider_group_id"])
    provider_dict["npi_list"].append(str(p["provider_groups"][0]["npi"]))
    provider_dict["tin"].append(p["provider_groups"][0]["tin"]["value"])
    provider_dict["tin_type"].append(p["provider_groups"][0]["tin"]["type"])
    provider_row = pd.DataFrame.from_dict(provider_dict)
    provider_df = pd.concat([provider_df, provider_row])
provider_df = provider_df.reset_index(drop=True)

In [65]:
provider_df.to_csv("../../data/aetna_piecewise/provider_references.csv")

## Extract price per procedure csv

In [5]:
def extract_dictlike(s):
    start_idx = s.find("negotiation_arrangement")
    end_idx = s.find("negotiated_rates")
    first_dictlike = s[start_idx-1:end_idx-2]
    first_dict = "{" + first_dictlike + "}"
    first_dict = json.loads(first_dict)
    return first_dict

In [14]:
def add_row_per_modifier(df_row_base, prices, df, idx):
    df_row_base["provider_references"] = str(prices["provider_references"])
    for modified_rate in prices["negotiated_prices"]:
        df_row_base["negotiated_rate"] = [modified_rate["negotiated_rate"]]
        df_row_base["expiration_date"] = [modified_rate["expiration_date"]]
        df_row_base["billing_class"] = [modified_rate["billing_class"]]
        df_row_base["billing_code_modifier"] = [modified_rate.get("billing_code_modifier", None)]
        df_row = pd.DataFrame(df_row_base)
        df = pd.concat([df, df_row])
    return df

In [11]:
def update_df(fileobj, df_row_base, prices_df, num_iters=1000):
    for i in range(num_iters):
        line = next(f)
        if line.find("negotiation_arrangement") != -1:
            procedure_row = extract_dictlike(line)
            df_row_base.update(procedure_row)
        else:
            start_idx = line.find("provider_references")
            line = line[start_idx:]
            dictlike = '{"'+ line
            prices = json.loads(dictlike)
            prices_df = add_row_per_modifier(df_row_base, prices, prices_df, i)
    return prices_df

In [ ]:
df_row_base = {"negotiation_arrangement": [None], "name": [None], "billing_code_type": [None], "billing_code_type_version": [None], "billing_code": [None], "billing_code_modifier": [None],
              "description": [None], "negotiated_rate": [None], "expiration_date": [None], "provider_references": [None], "negotiated_type": [None], "billing_class": [None]}

df_row = dict(df_row_base)
prices_df = pd.DataFrame()
silver_path = "../../data/2022-09-05_bf1ee0a7-5440-47e6-a69f-c376fb6b01a7_Aetna-Life-insurance-Company.json"
with open(silver_path, 'r+') as f:
    write_iters = 1000
    read_line_iters = 1000
    start = 0
    for _ in range(start):
        next(f)
        
    for write_iter in range(write_iters):
        
        read_until = start+((write_iter+1)*read_line_iters)
        read_until_prev = start+((write_iter)*read_line_iters)
        prev_file = f"../../data/aetna_piecewise/procedure_costs_{start}_{read_until_prev}.csv"
        if os.path.exists(prev_file):
            prev_df = pd.read_csv(prev_file)
        else:
            prev_df = pd.DataFrame()
            
        now_df = pd.DataFrame()
        now_df = update_df(f, df_row_base, prices_df, read_line_iters)
        now_df = pd.concat([prev_df, now_df])
        now_df = now_df.loc[:, ~now_df.columns.str.contains('^Unnamed')]
        now_df.to_csv(f"../../data/aetna_piecewise/procedure_costs_{start}_{read_until}.csv")
        if os.path.exists(prev_file):
            os.remove(prev_file)
            

/tmp/ipykernel_6804/4277747077.py:20: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  prev_df = pd.read_csv(prev_file)
/tmp/ipykernel_6804/4277747077.py:20: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  prev_df = pd.read_csv(prev_file)
/tmp/ipykernel_6804/4277747077.py:20: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  prev_df = pd.read_csv(prev_file)
/tmp/ipykernel_6804/4277747077.py:20: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  prev_df = pd.read_csv(prev_file)
/tmp/ipykernel_6804/4277747077.py:20: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  prev_df = pd.read_csv(prev_file)
/tmp/ipykernel_6804/4277747077.py:20: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=Fals

In [ ]:
df = pd.read_csv("../../data/aetna_piecewise/debug/procedure_costs_41000_43000.csv")

In [157]:
len(df)
df.name.value_counts()

REPAIR CLAVICLE DISLOCATION                                   1698
MTHFR GENE                                                     400
ANES UPR LWR GI NDSC PX                                        151
IP Detox and ICD10DX F13.21                                    129
P32 NA PHOSPHATE                                                86
PART HOSP-PSYCH REV 912                                         19
INSERT RADIOACT ELEM IN LOW EXTREM SUBCU/FASCIA, OPEN            9
INSERTION OF RADIOACT ELEM INTO L UP EXTREM, PERC APPROACH       9
DILATION OF LEFT TEMPORAL ARTERY, PERC ENDO APPROACH             7
RESECTION OF CONDUCTION MECHANISM, OPEN APPROACH                 7
BYPASS R EXT ILIAC ART TO L FEMOR A W SYNTH SUB, OPEN            6
Observation Services - Beh..                                     5
Name: name, dtype: int64

In [161]:
subset = df.loc[df.name == "REPAIR CLAVICLE DISLOCATION"]
subset.provider_references.value_counts()
subset.loc[subset.provider_references == "[688181]"]

685 1698


,Unnamed: 0,negotiation_arrangement,name,billing_code_type,billing_code_type_version,billing_code,billing_code_modifier,description,negotiated_rate,expiration_date,provider_references,negotiated_type,billing_class
2772,2772,ffs,REPAIR CLAVICLE DISLOCATION,CPT,2021.0,23530,55,REPAIR CLAVICLE DISLOCATION,115.17,9999-12-31,[688181],NaN,professional
2773,2773,ffs,REPAIR CLAVICLE DISLOCATION,CPT,2021.0,23530,53,REPAIR CLAVICLE DISLOCATION,153.56,9999-12-31,[688181],NaN,professional
2774,2774,ffs,REPAIR CLAVICLE DISLOCATION,CPT,2021.0,23530,52,REPAIR CLAVICLE DISLOCATION,383.90,9999-12-31,[688181],NaN,professional
2775,2775,ffs,REPAIR CLAVICLE DISLOCATION,CPT,2021.0,23530,78,REPAIR CLAVICLE DISLOCATION,537.46,9999-12-31,[688181],NaN,professional
2776,2776,ffs,REPAIR CLAVICLE DISLOCATION,CPT,2021.0,23530,54,REPAIR CLAVICLE DISLOCATION,575.85,9999-12-31,[688181],NaN,professional
2777,2777,ffs,REPAIR CLAVICLE DISLOCATION,CPT,2021.0,23530,56,REPAIR CLAVICLE DISLOCATION,76.78,9999-12-31,[688181],NaN,professional
2903,2903,ffs,REPAIR CLAVICLE DISLOCATION,CPT,2021.0,23530,30,REPAIR CLAVICLE DISLOCATION,52.00,9999-12-31,[688181],NaN,professional


In [159]:
subset = df.loc[df.name == "MTHFR GENE"]
df.provider_references.value_counts()

162 400


[451624]            34
[337195]            23
[552545]            23
[293883]            23
[701845]            23
                    ..
[27626]              1
[1295]               1
[307283]             1
[240649, 411175]     1
[587278, 726614]     1
Name: provider_references, Length: 898, dtype: int64

In [29]:
df.loc[df.provider_references == "[451624]"]

,Unnamed: 0,negotiation_arrangement,name,billing_code_type,billing_code_type_version,billing_code,description,negotiated_rate,expiration_date,provider_references,negotiated_type,billing_class
900,900,ffs,P32 NA PHOSPHATE,HCPCS,2021.0,A9563,P32 NA PHOSPHATE,365.85,9999-12-31,[451624],NaN,institutional
923,923,ffs,P32 NA PHOSPHATE,HCPCS,2021.0,A9563,P32 NA PHOSPHATE,300.00,9999-12-31,[451624],NaN,institutional
1898,1898,ffs,P32 NA PHOSPHATE,HCPCS,2021.0,A9563,P32 NA PHOSPHATE,365.85,9999-12-31,[451624],NaN,institutional
1921,1921,ffs,P32 NA PHOSPHATE,HCPCS,2021.0,A9563,P32 NA PHOSPHATE,300.00,9999-12-31,[451624],NaN,institutional
3886,3886,ffs,P32 NA PHOSPHATE,HCPCS,2021.0,A9563,P32 NA PHOSPHATE,365.85,9999-12-31,[451624],NaN,institutional
3909,3909,ffs,P32 NA PHOSPHATE,HCPCS,2021.0,A9563,P32 NA PHOSPHATE,300.00,9999-12-31,[451624],NaN,institutional
4884,4884,ffs,P32 NA PHOSPHATE,HCPCS,2021.0,A9563,P32 NA PHOSPHATE,365.85,9999-12-31,[451624],NaN,institutional
4907,4907,ffs,P32 NA PHOSPHATE,HCPCS,2021.0,A9563,P32 NA PHOSPHATE,300.00,9999-12-31,[451624],NaN,institutional


In [21]:
df_row_base = {"negotiation_arrangement": [None], "name": [None], "billing_code_type": [None], "billing_code_type_version": [None], "billing_code": [None],
              "description": [None], "negotiated_rate": [None], "expiration_date": [None], "provider_references": [None], "negotiated_type": [None], "billing_class": [None]}

df_row = dict(df_row_base)

In [22]:
df_row.keys()

dict_keys(['negotiation_arrangement', 'name', 'billing_code_type', 'billing_code_type_version', 'billing_code', 'description', 'negotiated_rate', 'expiration_date', 'provider_references', 'negotiated_type', 'billing_class'])

In [23]:
df_row = df_row_base.copy()
procedure_dict = dict(df_row)
iter_step_size = 1000000

## Extract the disease and pricing information for 10 iter cycles

In [63]:
iter_step_size = 1000000
for i in range(10):
    main = iterate_for_df(parser, df_row, iter_step_size)
    main.to_csv(f"../../data/aetna_piecewise/california_sept_{i}.csv")

UnboundLocalError: local variable 'provider_references_list' referenced before assignment